In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
import math

In [2]:
# Opening JSON file
f = open('twitter_aapl_new.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [3]:
flataapl  = [val for sublist in aapl for val in sublist]

In [4]:
print(flataapl[0])

{'text': 'RT @DomainFQ: .\nDomain Name for sale\n\nhttps://t.co/f056WpEdXS\n\n#fluke #FlukeTV #domain #100DaysOfCode #javascript #python #Apple #vr #Micro…', 'referenced_tweets': [{'type': 'retweeted', 'id': '1391353131334717440'}], 'id': '1392083055460851716', 'lang': 'en', 'created_at': '2021-05-11T11:43:58.000Z'}


In [5]:
df = pd.DataFrame(flataapl)
df

,text,referenced_tweets,id,lang,created_at,withheld
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,"[{'type': 'retweeted', 'id': '1391353131334717...",1392083055460851716,en,2021-05-11T11:43:58.000Z,NaN
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083046459969541,en,2021-05-11T11:43:56.000Z,NaN
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,"[{'type': 'retweeted', 'id': '1391353349669154...",1392083045058940935,en,2021-05-11T11:43:56.000Z,NaN
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083027929358336,en,2021-05-11T11:43:52.000Z,NaN
4,🍎 Apple Air Tags\nCALL FOR PRICE\nMOQ APPLY\nC...,NaN,1392083005594873856,en,2021-05-11T11:43:46.000Z,NaN
...,...,...,...,...,...,...
3390449,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11T13:00:06.000Z,NaN
3390450,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11T13:00:03.000Z,NaN
3390451,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11T13:00:03.000Z,NaN
3390452,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11T13:00:02.000Z,NaN


In [6]:
y = df['withheld'].dropna()
y

111581     {'copyright': False, 'country_codes': ['IN']}
169202     {'copyright': False, 'country_codes': ['IN']}
199596     {'copyright': False, 'country_codes': ['IN']}
304544     {'copyright': False, 'country_codes': ['IN']}
317803     {'copyright': False, 'country_codes': ['IN']}
                               ...                      
3264756    {'copyright': False, 'country_codes': ['IN']}
3264931    {'copyright': False, 'country_codes': ['IN']}
3265032    {'copyright': False, 'country_codes': ['IN']}
3265033    {'copyright': False, 'country_codes': ['IN']}
3352769    {'copyright': False, 'country_codes': ['IN']}
Name: withheld, Length: 125, dtype: object

In [7]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,text,referenced_tweets,id,lang,created_at
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,"[{'type': 'retweeted', 'id': '1391353131334717...",1392083055460851716,en,2021-05-11T11:43:58
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083046459969541,en,2021-05-11T11:43:56
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,"[{'type': 'retweeted', 'id': '1391353349669154...",1392083045058940935,en,2021-05-11T11:43:56
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083027929358336,en,2021-05-11T11:43:52
4,🍎 Apple Air Tags\nCALL FOR PRICE\nMOQ APPLY\nC...,NaN,1392083005594873856,en,2021-05-11T11:43:46
...,...,...,...,...,...
3390449,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11T13:00:06
3390450,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11T13:00:03
3390451,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11T13:00:03
3390452,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11T13:00:02


In [8]:
print(type(df1['referenced_tweets'][3390453]))
print(isinstance(df1['referenced_tweets'][4], float))

<class 'float'>
True


In [9]:
dfsmall = df1.iloc[0:10]
dfsmall
df2 = df1

In [90]:
d = df1.loc[df1['text'] == str("It's no secret that Apple products are pricey")]
g = df1.loc[df1['id'] == str(1392055958390587393)]
print(d)
g.iloc[0,0]
df1['id'].isnull().values.any()

Empty DataFrame
Columns: [text, referenced_tweets, id, lang, created_at]
Index: []


IndexError: single positional indexer is out-of-bounds

In [ ]:
for index, row in df1.iterrows():
#     a = row['referenced_tweets']
    if isinstance(df1['referenced_tweets'][index], float) == False:
        rtweetid = df1['referenced_tweets'][index][0]['id']
#         print(rtweetid)
        if (df1.loc[df1['id'] == str(rtweetid)]).empty == False:
            originalrow = df1.loc[df1['id'] == str(rtweetid)]
            df2['text'][index] = originalrow.iloc[0,0]
#         print(originalrow)
#         print(originalrow.iloc[0,0])

df2

In [ ]:
# print(df1['referenced_tweets'][0][0]['id'])
# print(type(df1['id'][0]))

for index, row in df1.iterrows():
#     a = row['referenced_tweets']
    if isinstance(df1['referenced_tweets'][index], float) == False:
        rtweetid = df1['referenced_tweets'][index][0]['id']
        originalrow = df1.loc[df1['id'] == rtweetid]
        df1['text'][index] = originalrow['text']

In [60]:
df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [2]:
df1 = pd.read_csv("twitter_aapl_1_clean.csv")
# Only including this step for now, basically removes the time component, leaving only date
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-9])
df1 = df1.rename(columns={"created_at" : "Date"})
df1

,Date,id,text
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [3]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend
def if_weekend(d = datetime.today()):
    if d.weekday() == 5:
        da = d + timedelta(days=2)
        return da
    elif d.weekday() == 6:
        da = d + timedelta(days=1)
        return da
    elif d == date(2021,4,2):
        return date(2021,4,5)
    else:
        return d
print(if_weekend(date(2021,6,14)))
print(type(date(2021,6,14)))

2021-06-14
<class 'datetime.date'>


In [4]:
df1['Date']= pd.to_datetime(df1['Date'])
df1['usedate'] = df1['Date'].map(lambda x: if_weekend(x))
df1

,Date,id,text,usedate
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11
...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18


In [5]:
price = pd.read_csv("aapl2.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,125.9100,126142800,123.5000,126.2700,122.7700,2.4100
1,2021-05-10,126.8500,88071230,129.4100,129.5400,126.8100,-2.5600
2,2021-05-07,130.2100,78973270,130.8500,131.2582,129.4750,-0.6400
3,2021-05-06,129.7400,78128330,127.8900,129.7500,127.1300,1.8500
4,2021-05-05,128.1000,84000900,129.2000,130.4500,127.9700,-1.1000
...,...,...,...,...,...,...,...
247,2020-05-18,78.7400,135372520,78.2925,79.1250,77.5810,0.4475
248,2020-05-15,76.9275,166348360,75.0875,76.9750,75.0525,1.8400
249,2020-05-14,77.3850,158929080,76.1275,77.4475,75.3825,1.2575
250,2020-05-13,76.9125,200622560,78.0375,78.9875,75.8025,-1.1250


In [6]:
merged = pd.merge(df1, price, how='left', on='usedate')
merged

,Date,id,text,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
...,...,...,...,...,...,...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51


In [7]:
merged.isna().sum()

Date          0
id            0
text          0
usedate       0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
Change        0
dtype: int64

In [24]:
# merged.to_csv("merged.csv", index = False)